In [1]:
!pip install statsmodels --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 4.7 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
from sqlalchemy import create_engine
import numpy as np
import matplotlib
from time import time
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta, date, datetime
import matplotlib.ticker as tkr
from scipy import stats
plt.style.use('fivethirtyeight')
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
matplotlib.rcParams['text.color'] = 'k'
from pylab import rcParams
from statsmodels.tsa.seasonal import STL

%matplotlib inline

In [3]:
from google.colab import files
import io
uploaded = files.upload()

Saving Global DataBase3.csv to Global DataBase3.csv
Saving Global DataBase2 daily.csv to Global DataBase2 daily.csv


In [32]:
#read dataframe from DB
df=pd.read_csv(io.BytesIO(uploaded['Global DataBase2 daily.csv']), index_col='datetime_per_day', parse_dates=True, dayfirst=True)

df2=pd.read_csv(io.BytesIO(uploaded['Global DataBase3.csv']), index_col='datetime', parse_dates=True, dayfirst=True)


columns = ['Price - PT [€/MWh]','Price - ES [€/MWh]','Actual Consumption (MWh)','Biomass AA','Fossil Gas AA','Fossil Hard Coal AC', 'Hydro Pumped Storage AA','Hydro Pumped Storage AC','Hydro Run-of-river and poundage AA','Hydro Water Reservoir AA', 'Other AA', 'Solar AA','Wind Onshore AA']
for column in columns : 
    df[column] = df2[column].resample("D").mean()

columns = ['Electricity Consumption (MWh)']
for column in columns : 
    df[column] = df2[column].resample("D").sum()
    
df = df.sort_index()


#Handeling global outliers
GlobalOutliers =  []
for i in range(13):
    GlobalOutliers.append(df.idxmax()['Price - PT [€/MWh]'])
    df.at[df.idxmax()['Price - PT [€/MWh]'], 'Price - PT [€/MWh]'] = np.nan

#Replacing them with the mean of that day
for d in GlobalOutliers:
    sdt = datetime.combine(d, datetime.min.time())
    edt = sdt + timedelta(hours=24)
    df.at[d, 'Price - PT [€/MWh]'] = df[sdt:edt]['Price - PT [€/MWh]'].mean()
    
#Handling negative values 
negativeIndex = df.index[df['Price - PT [€/MWh]'] < 0]
for index in (negativeIndex):
    df.at[index, 'Price - PT [€/MWh]']= df.at[index - timedelta(hours=1), 'Price - PT [€/MWh]']
    
df = df.dropna()

In [33]:
df

,Price - PT [€/MWh],Price - ES [€/MWh],Actual Consumption (MWh),Biomass AA,Fossil Gas AA,Fossil Hard Coal AC,Hydro Pumped Storage AA,Hydro Pumped Storage AC,Hydro Run-of-river and poundage AA,Hydro Water Reservoir AA,Other AA,Solar AA,Wind Onshore AA,Electricity Consumption (MWh)
datetime_per_day,,,,,,,,,,,,,,
2017-01-01,51.42,51.05,4875.13,351.65,1138.83,0.00,137.17,402.09,405.70,80.65,40.00,56.61,651.48,112128
2017-01-02,58.88,59.92,4915.58,347.17,2182.42,0.00,226.96,337.83,535.38,59.08,39.38,17.62,3906.83,117974
2017-01-03,65.25,65.25,4915.58,344.04,2582.38,0.00,828.58,441.38,732.88,355.96,40.50,23.08,1877.00,117974
2017-01-04,66.70,66.52,4915.58,339.54,2573.00,0.00,580.71,134.25,653.75,278.62,38.33,36.58,207.08,117974
2017-01-05,66.86,64.48,4915.58,347.21,2172.12,6.33,527.79,97.50,855.75,326.08,41.42,42.33,549.88,117974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-01,259.46,259.46,4915.58,377.54,2270.04,0.00,238.67,294.21,272.88,60.00,32.67,229.42,1107.42,117974
2022-03-02,254.47,254.47,4915.58,414.08,2270.04,0.00,311.88,298.38,366.75,73.50,33.04,304.08,1362.62,117974
2022-03-10,284.07,376.24,4915.58,378.83,2159.67,0.00,345.79,485.50,404.00,55.58,26.83,277.62,2510.92,117974


In [34]:
# Split into train and test set
data = df[: '2021-11-01 00:00:00']
test_data = df['2021-11-01 00:00:00': ]

In [35]:
start_time = time()

fitted_model1 = ExponentialSmoothing(data["Price - PT [€/MWh]"],trend='add').fit()

exec_time1 = time()-start_time

In [36]:
test_predictions1 = fitted_model1.forecast(test_data.shape[0])

In [37]:
start_time = time()

fitted_model2 = ExponentialSmoothing(data['Price - PT [€/MWh]'],trend='add',seasonal='add',seasonal_periods=12).fit()

exec_time2 = time()-start_time

In [38]:
test_predictions2 = fitted_model2.forecast(test_data.shape[0])

In [39]:
from sklearn import metrics 
#Calculate MAE, MSE, RMSE, CV DAY
MAE= metrics.mean_absolute_error(test_data['Price - PT [€/MWh]'], test_predictions1)
MSE=metrics.mean_squared_error(test_data['Price - PT [€/MWh]'], test_predictions1)
CV= (np.sqrt(metrics.mean_squared_error(test_data['Price - PT [€/MWh]'], test_predictions1))/test_data['Price - PT [€/MWh]'].mean())*100
R2= metrics.r2_score(test_data['Price - PT [€/MWh]'], test_predictions1)


print('*************** Exponential Smoothing Results ***************')
print('Mean Absolute Error:', MAE)
print('Mean Squared Error:', MSE)  
print('Root Mean Squared Error:', np.sqrt(MSE))
print('Coefficient of Variance:',CV)
print('R2:', R2)
print('Execution Time:', '%.5f' %  exec_time1)

*************** Exponential Smoothing Results ***************
Mean Absolute Error: 117.17111995181064
Mean Squared Error: 15933.051283779867
Root Mean Squared Error: 126.22619095805699
Coefficient of Variance: 60.545290145461585
R2: -6.195252524214158
Execution Time: 0.06808


In [40]:
from sklearn import metrics 
#Calculate MAE, MSE, RMSE, CV DAY
MAE= metrics.mean_absolute_error(test_data['Price - PT [€/MWh]'], test_predictions2)
MSE=metrics.mean_squared_error(test_data['Price - PT [€/MWh]'], test_predictions2)
CV= (np.sqrt(metrics.mean_squared_error(test_data['Price - PT [€/MWh]'], test_predictions2))/test_data['Price - PT [€/MWh]'].mean())*100
R2= metrics.r2_score(test_data['Price - PT [€/MWh]'], test_predictions2)


print('*************** Holt Winter Results ***************')
print('Mean Absolute Error:', MAE)
print('Mean Squared Error:', MSE)  
print('Root Mean Squared Error:', np.sqrt(MSE))
print('Coefficient of Variance:',CV)
print('R2:', R2)
print('Execution Time:', '%.5f' %  exec_time2)

*************** Holt Winter Results ***************
Mean Absolute Error: 118.14494588931639
Mean Squared Error: 16169.556563010372
Root Mean Squared Error: 127.15957125993455
Coefficient of Variance: 60.99299264495326
R2: -6.302056624512614
Execution Time: 0.41281
